# TODO 05/07 FINISHD SECTION ANALYZE DATA
# TODO: APPLY COMBINE CROSS VALIDATION WITH EVALUTATE FUNCTION
# no use lasso regularization now
# Relu as activation function in hidden layer works better than sigmoid
# The most powerful NN setting:
1. only 1 hidden layer
2. 2000 neurons
3. learning rate: 0.1
# create double hidden layer NN class: -> complete autoencoder today, 5.04 V


---------


---------
# NO MACHINE LEARNING LIBARY USED FOR TRAINING/REGRESSION/CLASSFICATION
# sklean.feature_extraction ONLY USED for language data into Vector transformation
# preprocess dataset

In [1]:
import pandas as pd
import csv
import codecs
import numpy as np
import matplotlib.pyplot as plt
import os,sys
from sklearn.feature_extraction.text import CountVectorizer#ONLY USED FOR TRANSFORM FROM LANGUAGE TO VECTOR
from numpy import linalg as LA
import math

In [2]:
df = pd.read_csv('./Tab.delimited.Cleaned.dataset.WITH.variable.labels.csv', sep='\t',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (17,55,59,61,65,68,69,70,83,90,91,92,93,120,121,122,123,126,140,141) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df3 = pd.read_csv('./ML3AllSites.csv', sep=',',encoding='ISO-8859-1')

/usr/lib/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1,11,12,19,20,129,132,169,230) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
class NeuralNetwork:
    def __init__(self, n_h, n_iterate=10, learning_rate=1, dic = []):
        self.n_x = None  # size of the input layer
        self.n_h = n_h  # size of the hidden layer
        self.n_y = None # size of the output layer
        self.W1 = None
        self.W2 = None
        self.b1 = None
        self.b2 = None
        self.A1 = None
        self.A2 = None  # sigmoid output of the second activation
        self.learning_rate = learning_rate
        self.n_iterate = n_iterate
        self.lamda = 0.01
#         self.dic = dic
        self.dic = dic
    
    def initialize_parameters(self):
        self.W1 = np.random.randn(self.n_h, self.n_x) * 0.01
        self.b1 = np.zeros((self.n_h, 1))
        self.W2 = np.random.randn(self.n_y, self.n_h) * 0.01
        self.b2 = np.zeros((self.n_y, 1))
       
    def MSE(self,Y):
        cost = 0
        for i in range(len(Y)):
            for j in range(len(Y[0])):
                cost+=(Y[i][j]-self.A2[i][j])**2
        cost = cost/len(Y)
        return cost
        
    def relu(self, z):
        return z * (z > 0)
    
#     def softmax(self,z):
#         exps = np.exp(z-np.max(z,axis= 1,keepdims = True))
#         return exps/np.sum(exps,axis = 1, keepdims = True)
    
    

#     def softmax(self,A):  
#         expA = np.exp(A)
#         return expA / expA.sum(axis=1, keepdims=True)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    
    def cro_entro(self,Y):
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean()
        return cost
    
    #changed for mult layer
    #add lasso regularization 
    def compute_cost(self, Y):# cross entrophy
        #cost = np.linalg.norm(self.A2 - Y)
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean() 
#         print('shape cost is: ' ,cost.shape)

#         <----lasso regularization ---->
#         tmpcost = self.A2.T
#         tmpcost = np.sum(tmpcost,axis = 1)/len(Y)
#         <----End of lasso regularization implementation ----- >
        return np.squeeze(cost)

    def forward_propagation(self, X):
        self.A1 = self.relu(self.W1 @ X + self.b1)
        #dropout function for output in hidden layer 
        u1 = np.random.binomial(1,0.5,size = self.A1.shape)
        self.A1 *= u1
    
        self.A2 = self.sigmoid(self.W2 @ self.A1 + self.b2)

#         u2 = np.random.binomial(1,0.5,size = self.A2.shape)
#         self.A2 *= u2
#         self.A2 = self.soft_max_For_multi_attribute(self.A2,self.dic)

        
    
    def backward_propagation(self, X, Y):
        m = X.shape[1]
#       <------dz2 for lasso regression ----- > 
#         dZ2 = (self.A2 - Y)/m  +self.lamda/m
#       <------dz2 for lasso regression ----- > 

        dZ2 = (self.A2 - Y)/m
        dW2 = dZ2 @ self.A1.T 
        db2 = np.sum(dZ2, axis=1, keepdims=True) 
        dZ1 = self.W2.T @ dZ2 * (self.A1 > 0)
        dW1 = dZ1 @ X.T 
        db1 = np.sum(dZ1, axis=1, keepdims=True) 

        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2

    def fit(self, X, Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
        print('n_x is',self.n_x ,'n_y is ',self.n_y )
        self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                  self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))

    def fit_continue(self,X,Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
#         if self.W1 is None:
#             self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                 self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))
                
    def predict(self, X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)

        
        A2 = self.sigmoid(self.W2 @ A1 + self.b2)
        tmp = np.dot(self.W2,self.A1)
    
        return A2.T
    
    def predict_one_layer(self,X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)
        return A1.T
    
    def soft_max_For_multi_attribute(self,arr,dic):
        #create array with all dict keys based on insertion order
        tmp_arr = np.copy(arr)
        
        dicarr = list(dic.keys())
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length    
            #find the targeted array need to perform softmax
            #create a dic to contain the location of those onehotencoding attribute
            loc_dic = []
            for i in range(cur_dict_length-1):
                loc_dic.append(cur_loc+i)
            #find targed array
            target = tmp_arr[:,loc_dic]
            #get softmaxed arr
            arr_soft_max = self.api_softmax(target)
            #reassig value back to orignal arr
            tmp_arr[:,loc_dic] = arr_soft_max
            #update cur_loc
            cur_loc +=cur_dict_length
            
        return tmp_arr

    def api_softmax(self,z):
        exps = np.exp(z)
        return exps/np.sum(exps,axis = 1, keepdims = True)

In [5]:
class KMeans:
    def __init__(self, n_clusters=64):
        self.n_clusters = n_clusters  # number of clusters
        self.centers = None  # to record the centers
        self.labels = None
        self.Y = None

    def random_center(self,Y): #only for 1d dataset
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
            
            
    def random_center2(self,Y):
    # randomly generate n_cluster clusters in the raange of X
        self.centers = np.random.rand(self.n_clusters, len(Y[0]))
        for i in range(self.n_clusters):
            self.centers[i] = Y[i]
#         for j in range(0):
#             Y_j_min = self.Y[:,j].min()
#             Y_j_max = self.Y[:,j].max()
#             self.centers[:,j] = Y_j_min + (Y_j_max - Y_j_min) * self.centers[:,j]
            
            
            
    def dist(self, point1, point2): #old one
        return 2*(point1[0]-point2[0])**2 + 4*(point1[1]-point2[1])**2 + 3*(point1[2]-point2[2])**2

    def dist2(self,point1,point2):
        return LA.norm(point1-point2)**2
    
    def fit(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    def fit2(self, Y):
        self.Y = Y
        self.labels = np.zeros(Y.shape[0], dtype='uint8')  # record the current labels of each sample of X
        self.random_center2(Y)
        diff = 1
        
        while diff > 1e-3:
            old_center = self.centers.copy()

            # go through all samples and label them using the nearest label
            for i in range(Y.shape[0]):
                distance = np.zeros(self.n_clusters)
                for j in range(self.n_clusters):
                    distance[j] = self.dist2(Y[i], self.centers[j])
                self.labels[i] = np.argmin(distance)
                
                
            # update the centers
            for i in range(self.n_clusters):
                self.centers[i] = Y[self.labels==i].mean(axis=0)
                

            # update the difference
            diff = np.linalg.norm(self.centers - old_center)
            print(diff)
        return self
    
    
    def transform(self, Y):
        out = np.zeros(Y.shape)
        for i in range(self.n_clusters):
            out[self.labels==i] = self.centers[i]
        return out
    

# oneHOTENCODING

In [6]:
def oneHotEncoding(Y, kmeans):
    out = np.zeros((len(Y), kmeans.n_clusters))
    for i in range(len(Y)):
        out[i, Y[i]] = 1
    return out

# test clustering numbered data:
    step1 : get single column
    step2 : sort in increasing order
    step3 : based on distribution, cut data into 4 section with equally number of candidate
    step4 : if data is null, set isnon to be true for that column
    step5 : append new datacol to input_arr

In [7]:
def isint(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

def isfloat(value):
  try:
    float(value)
    return True
  except ValueError:
    return False

def getdistriarr(df,arr_str):
    arr = df[arr_str].values
    di = []
    di_og = []  #RETURN OG SINCE di will be sort
    for i in range(len(arr)):
        if isint(arr[i]) == True:
            di.append(int(arr[i]))
            di_og.append(int(arr[i]))
        else:
            di.append(-1)
            di_og.append(-1)
    di.sort()
    ans_arr = []
    leng = len(di)
    ans_arr.append(di[int(leng/3)])
#     ans_arr.append(di[int(leng/2)])
    ans_arr.append(di[int(leng*0.66)])
    ans_arr.append(di[int(leng-1)])
    return di_og,ans_arr

def oneHotEncoding_return_arr(att_arr,class_arr): #one 
    #add one more column for each row since last col used as flag
    choice = len(class_arr)
    out = np.zeros((len(att_arr),len(class_arr)+1))
    for i in range(len(att_arr)):
        loc = 0
        #first jude if data is null
        if att_arr[i] == -1:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            for j in range(len(class_arr)):
                if att_arr[i] == class_arr[j]:
                    out[i][j] = 1
                    break
        
    return out


def GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,attr_name,og_dic):
    att_arr,class_arr = getdistriarr(df,attr_name)
    dic_ans = create_dic_for_classification(class_arr)
    append_arr_of_dic_to_overall(og_dic,dic_ans,attr_name)

    return oneHotEncoding_return_arr(att_arr,class_arr)

def create_dic_for_classification(class_arr):
    cur_dic = {}
    total_len = len(class_arr)
    for i in range(len(class_arr)):
        cur_dic[class_arr[i]] = i
    return cur_dic

def append_arr_of_dic_to_overall(og,dic,att_name):
    og[att_name] = dic
    
    
def append_arr(old,new):
    #both old and new has the same num of row
    #create a new nparray
    
    newdim = len(old[0])+len(new[0])
    ans = np.zeros((len(old),newdim))
    print(ans.shape)
    for i in range(len(old)):
        ans[i] = np.append(old[i],new[i])
    return ans

# STEP TWO CONVERT NLP DATASET INTO VECTOR:
    STEP1: vectorize dataset
    STEP2: apply kmeans clustering to language data
    STEP3: ONEHOT ENCODING
    STEP4: APPEND CLASSIFICATION DATASET INTO old CLASSFICAITION ARR

In [8]:
def NLPDATAPRO(allsentences):
    vectorizer = CountVectorizer()
    X = vectorizer.fit_transform(allsentences)
    ans = X.toarray()
    return ans

def createtextarr_ONEHOT_ENCODING(attri_col): #ONEHOT_ENCODING
    df2 = df[attri_col]
    arr = df2.values
    for i in range(len(arr)):
        di.append(arr[i])
    return arr

# STEP THREE CONVERT TEXT_DATASET WITH FIXED classification into vector

In [9]:
def processnull(arr):
    arrnew = []
    for i in range(len(arr)):
        if type(arr[i]) == float and math.isnan(arr[i]) :
            arrnew.append('Nan')
        else:
            arrnew.append(arr[i])
    return arrnew

def returnuniqiearr(og):
    ar = np.asarray(og)
    return np.unique(ar)

# TODO: create a function to append text to old vecto(remember to append flag)

# TODO: TEST NN TONIGHT?

In [242]:
def create_onehoted_text_fixed_classification(attribu_name,df,km_num):
    df_cur = df[attribu_name]
    attribute_arr = df_cur.values
    
    arr_no_null = processnull(attribute_arr) #create new arr with format fit by NLP(CountVectorizer)
   
    NLP_ARR = NLPDATAPRO(arr_no_null)
    #BEFORE ONEHOT_ENCODING
    #APPLY KMEANS CLUSTERING SINCE TOO MANY DIFFERENT CLASSIFICATION
    KM_TMP = KMeans(km_num)
    KM_TMP.fit2(NLP_ARR)
    labels_arr = KM_TMP.labels
    
    return labels_arr



def onehot_enco(labelarr,df,att_name):
    og_text_arr = df[att_name].values
    #create dic to remeber loc for each value in the uniqe arr
    uni_arr = np.unique(labelarr)
    loc_dic = {}
    for i in range(len(uni_arr)):
        loc_dic[uni_arr[i]] = i
    #based on the lenth of dic,create onehot_encod
    choice = len(loc_dic)
    out = np.zeros((len(labelarr),choice+1))
    for i in range(len(labelarr)):
        #first junde if data is null
        if type(og_text_arr[i])!=str and math.isnan(og_text_arr[i]) == True:
            out[i][-1] = 0
            continue
        else:
            out[i][-1] = 1
            cur_loc = loc_dic.get(labelarr[i])
            out[i][cur_loc] = 1
            
    return out,loc_dic

def GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,attr_name,og_dic,km_num):
    labels_arr = create_onehoted_text_fixed_classification(attr_name,df,km_num)
    out_arr,loc_dic = onehot_enco(labels_arr,df,attr_name)
    #append dic to global array of dic
    #append reverse dic to overall
#     dicnew = reversedict(loc_dic)
    append_arr_of_dic_to_overall(og_dic,loc_dic,attr_name)
    p = list(np.unique(labels_arr))
    print('class_arr of nlp', p)
    return out_arr
#     return out_arr   

# def reversedict(dic):
#     dicnew = {}
#     keyarr = list(dic.keys())
#     valarr = list(dic.values())
#     for i in range(len(keyarr)):
#         dicnew[valarr[i]] = keyarr[i]
#     return dicnew

# GENERAL WORK:
    1. MAINTAIN AN ARRAY OF DICTIONARY S.T EVERY DIC
       CONTAINS MAPPING FROM CLASSIFICAITON TO ACTUAL INDEX AFTER KMEANS
    
    2. AFTER TRAINING, WE WILL UTILIZED THE ARR_DIC TO RECOVER CANDIDATE ANSER
    
    3. TODO: REWRITE COST FUNCTION OF NERUAL NETWORK
    

# FINDING:
    AFTER combine several attributes column, num of candidate with dinct
    ans grows extremly fast.
    Need to apply Kmeans clustering again to the array of candidate before
    threw into Neural network

# CREATE A SCRIPT TO APPEND ASSIGNED ATTRIBUTE TO INPUT COLUMN

(API)INSTRUCTION:<br>
USER WHO WANTS TO CREATE PREPROCESSED DATASET ONLY NEED TO CREATE
ARRAY[i]<br>
       ARRAY[i][0] = 'name of attribute'<br>
       ARRAY[i][1] = 1 : it is a digit column<br>
       ARRAY[i][1] = 0 : it is a NLP column



In [200]:
def semi_auto_append_attri(arr_of_attribute,df,og_dic):
    for i in range(len(arr_of_attribute)):
        name = arr_of_attribute[i][0]
        isdigit = arr_of_attribute[i][1]
        if isdigit == 1:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,name,og_dic)
        else:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,name,og_dic,80)
        #append old with new
        if i == 0:
            old = attr_arr
        else:
            old = append_arr(old,attr_arr)
#             print(old.shape)
    return old
        

In [235]:
def new_semi_auto_append_attri(arr_of_attribute,df,og_dic):
    for i in range(len(arr_of_attribute)):
        name = arr_of_attribute[i][0]
        print('cur key is', name)
        isdigit = arr_of_attribute[i][1]
        if isdigit == 1:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_DIGIT_COLUMN(df,name,og_dic)
        elif isdigit == 0:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_NLP_COLUMN(df,name,og_dic,30)
        elif isdigit == 2:
            attr_arr = GET_ONE_HOT_ARRAY_ONLY_FIXED_CHOICE_COLUMN(df,name,og_dic)
        #append old with new
        if i == 0:
            old = attr_arr
        else:
            old = append_arr(old,attr_arr)
#             print(old.shape)
    return old

In [13]:
def fix_choice_get_distinct_arr(df,arr_str):
    arr = df[arr_str].values
    uni_arr = np.unique(arr)
    di = []
#     di_og = [] #RETURN OG SINCE di will be sorted
    div_arr = []
    for i in range(len(uni_arr)):
        if math.isnan(uni_arr[i])!=True:
            div_arr.append(uni_arr[i])
    for i in range(len(arr)):
        if math.isnan(arr[i])!= True:
            di.append(int(arr[i]))
        else:
            di.append(-1)
    return di,div_arr

def GET_ONE_HOT_ARRAY_ONLY_FIXED_CHOICE_COLUMN(df,attr_name,og_dic):
    att_arr,class_arr = fix_choice_get_distinct_arr(df,attr_name)
    print('class_arr',class_arr)
    dic_ans = create_dic_for_classification(class_arr)
    append_arr_of_dic_to_overall(og_dic,dic_ans,attr_name)
    return oneHotEncoding_return_arr(att_arr,class_arr)

# Apply Kmeans Clustering to the overal column

In [14]:
def cluster_whole_attribute_get_Y_for_Neural_NetWork(X,num_cluster):
    km = KMeans(num_cluster)
    km.fit2(X)
    
    #APPLY ONEHOT_ENCODING TO NEURAL_NETWORKAGAIN
    
    return oneHotEncoding(km.labels,km),km

# APPLY NEURAL NETWORK TO SOLVE THIS QUESTION:

REMEBER TO CONSIDER FLAG FOR EACH COLUMN

In [15]:
def process_predicted_data(pred,kmeans):
    ans = []
    for i in range(len(pred)):
        ans.append(kmeans.centers[np.argmax(pred[i])])
    newarr = np.asarray(ans)
    return newarr

def find_classification_based_on_distance(pred,dic):
    #create array with all dict keys based on insertion order
    dicarr = list(dic.keys())
    #isolate target awated_array used to classification
    for i in range(len(pred)):
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
#             print('cur dic: ',cur_dict_key)
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_lengh = len(cur_dict_arr)+1
            cur_des += cur_dict_lengh  #update current cutting position
            #restore dimension of onehotencoding for current dictionary
            one_hot_cur_dic = np.zeros((cur_dict_lengh,cur_dict_lengh))
#             print('before any processing cur loc is ', cur_loc, ' cur des is : ', cur_des)
            for q in range(0,cur_dict_lengh-1):
                one_hot_cur_dic[q][q] = 1
                one_hot_cur_dic[q][-1] = 1
            #isolate target awaited_classification array
            cur_isolated_arr = pred[i][cur_loc:cur_des]
            #compute classification with smallest distance
            arr_store_distance= []
            for h in range (len(one_hot_cur_dic)):
                arr_store_distance.append(np.linalg.norm(cur_isolated_arr-one_hot_cur_dic[h]))
            #transfer dic to arr s.t unilized argmim
            dis_np_array = np.asarray(arr_store_distance)
            index = np.argmin(dis_np_array)
#             if cur_dict_key == 'big5_10':
#                 print(index)
#                 print('cur onehot_array is ', one_hot_cur_dic[3] )
#                 print('cur loc is ', cur_loc, ' cur des is : ', cur_des)
            # change value in output array
            for k in range (cur_dict_lengh):
                pred[i][cur_loc+k] = one_hot_cur_dic[index][k]
                
            cur_loc += cur_dict_lengh #update cur_loc not used in curretn loop

def find_classification_based_on_prob(pred,dic):
    dicarr = list(dic.keys())
    for i in range(len(pred)):
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_lengh = len(cur_dict_arr)  + 1
            cur_des += cur_dict_lengh
            
            cur_isolated_arr = pred[i][cur_loc:cur_des-1]
            #find index of max value
            index = np.argmax(cur_isolated_arr)
            
            for k in range(cur_dict_lengh):
                pred[i][cur_loc+k] = 0
            
            pred[i][cur_loc + index] = 1
            cur_loc +=cur_dict_lengh
    
            
def create_reverse_dictionary(dic):
    reverse = {}
    dic_arr = list(dic.keys())
    for i in range(len(dic_arr)):
        cur_reverse_dict = {}
        cur_dic_key = dic_arr[i]
        cur_key_arr = list(dic.get(cur_dic_key).keys())
        cur_index_arr = list(dic.get(cur_dic_key).values())
        #reverse key_value pair in originaly dictionary
        for j in range(len(cur_key_arr)):
            cur_reverse_dict[cur_index_arr[j]] = cur_key_arr[j]
        #append current dic to ans dic
        reverse[cur_dic_key] = cur_reverse_dict
    return reverse

# create  function to knock out some attribute inorder to create more dataset

In [251]:
def trainingset_generator(train,dic,arr_str):
    #first find the location 
    #knock out
    #return dataset
    dicarr = list(dic.keys())
    changed_arr = np.copy(train)
    y_arr = np.copy(train)
    #outermost loop : loop through those keys that need to be cleared
    for d in range(len(arr_str)):
        cur_key = arr_str[d]
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length
            #test if we find what we want
            if cur_dict_key == cur_key:
                for i in range(len(train)):
                    for k in range(cur_dict_length):
                        changed_arr[i][cur_loc + k] = 0
                        
                break
            cur_loc +=cur_dict_length
    return changed_arr,y_arr

def append_row(old,new):
    ans = np.vstack([old,new])
    return ans


def enlarge_dataset(train,y,dic,arr_of_arr_str):
    list_of_new_x = []
    list_of_new_y = []
    new_x = np.copy(train)
    new_y = np.copy(y)
    for i in range(len(arr_of_arr_str)):
        cur_str = arr_of_arr_str[i]
        changed_arr,y_arr = trainingset_generator(train,dic,cur_str)
        #append
        new_x = append_row(new_x,changed_arr)
        new_y = append_row(new_y,y_arr)
        list_of_new_x.append(changed_arr)
        list_of_new_y.append(y_arr)
#     return new_x,new_y
    return list_of_new_x,list_of_new_y

def createlist_list_string(dic):
    ans = []
    key_arr = list(dic.keys())
    for i in range(len(key_arr)):
        tmp = []
        tmp.append(key_arr[i])
        ans.append(tmp)
        
    #degree 2
#     for i in range(len(key_arr)):
#         for j in range(i,len(key_arr)):
#             tmp = []
#             tmp.append(key_arr[i])
#             tmp.append(key_arr[j])
#             ans.append(tmp)
    return ans

def train_list_of_dataset(nn,x,y):
    nn.fit(y,y)
    for i in range(0,len(x)):
        print('current iter', i)
        nn.fit_continue(x[i],y)
        #after training compute num of correctly predicted choice for candidate
        ans = nn.predict(x[i])
        find_classification_based_on_prob(ans,nn.dic)
        count_num_of_accurately_predicted_avg(ans,X_train_new,nn.dic)
    
    return nn
    
def count_num_of_accurately_predicted_avg(x,y,dic):
    dicarr = list(dic.keys())
    
    right_predicted_count = 0
    for i in range(len(x)):
        #iterate thourhgt each choice
        tmp_right_count = 0
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dic_length = len(cur_dict_arr) + 1
            cur_des += cur_dic_length
            issame = True
            #compare x and y in range cur_loc - > cur_des
            for pointer in range(cur_loc,cur_des-1):
                if x[i][pointer] != y[i][pointer]:
                    issame = False
                    break
            
            #now check whether x[i] and y[i] are same
            if issame == True:
                tmp_right_count+=1
            cur_loc += cur_dic_length
        #finished iterate throught current candidate
        right_predicted_count+=tmp_right_count
    
    print('fianl avg right predicted count is ', right_predicted_count/len(x))
    
    
    #function used to find how well the model interpolate
#need a array contains all attribute name
#already has the function to mask every attribute
#what is output? key value pair, key: attribute name, value: avg accuracy
def evaluate(X_train_arr,Y_train_arr,og_pred,att_name_arr,hidden_neuron,iter_num):
    evaluate_dic = {}
    #to do for every attribute array utlize cross validation
    #temporary: we split first 2500 as train
    for i in range(len(X_train_arr)):
        x_cur_all = X_train_arr[i]
        y_cur_all = Y_train_arr[i]
        cur_mask_key = att_name_arr[i][0]
        #split dataset into training
        x_cur_train = x_cur_all[:2400]
        y_cur_train = y_cur_all[:2400]
        x_cur_pred = x_cur_all[2400:]
        y_cur_pred = y_cur_all[2400:]
        #create nn object
        mm = NeuralNetwork(hidden_neuron,iter_num,0.1,og_pred)
        mm.fit(x_cur_train,y_cur_train)
        #predict the answer
        predict_ans = mm.predict(x_cur_pred)
        tmp_eval_ans = count_right_predicted_num(predict_ans,y_cur_pred,og_pred,cur_mask_key)
        evaluate_dic[str(cur_mask_key)] = tmp_eval_ans
        print('cur masked name is ', cur_mask_key, ' eval value is: ', tmp_eval_ans)
        
def count_right_predicted_num(ans,y_pred,dic,att_name):
    #first find the location of masked attribute name
    #loop through dictionary list
    anscount = 0
    totallen = len(ans)
    dicarr = list(dic.keys())
    cur_loc = 0
    cur_des = 0

    for j in range(len(dicarr)):
        cur_dict_key = dicarr[j]
#         print('curloop key', cur_dict_key)
        cur_dict_arr = dic.get(cur_dict_key)
        cur_dict_length = len(cur_dict_arr) + 1
        cur_des += cur_dict_length
        #find if cur key == att_name
        if cur_dict_key == att_name:
            break
        else:
            cur_loc +=cur_dict_length
#     print('!!!!curloc ', cur_loc, ' des ', cur_des)
    #now we have location of masked attribute column, we just need to examin the answer
    #first --softmax
    find_classification_based_on_prob(ans,og_pred)
    #second -> exame
    for i in range(totallen):
        issame = True
        xcur = ans[i][cur_loc:cur_des-1]
        ycur = y_pred[i][cur_loc:cur_des-1]
        if np.array_equal(xcur,ycur):
            anscount+=1
    print('current loc and des is', cur_loc, 'and ', cur_des, ' total cound is',  anscount,'percent is', anscount/totallen)
    return anscount/totallen
    
    
    
def create_dictionary_for_every_location_of_feature(dic):
    dic_look_up = {}
    dicarr = list(dic.keys())
    cur_loc = 0
    cur_des = 0 
    for i in range(len(dicarr)):
        locarr = []
        cur_dict_key = dicarr[i]
        cur_dict_arr = dic.get(cur_dict_key)
        cur_dict_length = len(cur_dict_arr)+1
        cur_des +=cur_dict_length
        #append loccation
        locarr.append(cur_loc)
        locarr.append(cur_des)
        dic_look_up[cur_dict_key] = locarr
        cur_loc  += cur_dict_length
    return dic_look_up

# Autoencoder - Double Hidden Layer Neural Network  Implementation

In [17]:
class MULTI_NeuralNetwork:
    def __init__(self, n_h,n_h2, n_iterate=10, learning_rate=1, dic = []):
        self.n_x = None  # size of the input layer
        self.n_h = n_h  # size of the hidden layer
        self.n_h2= n_h2
        self.n_y = None # size of the output layer
        self.W1 = None
        self.W2 = None
        self.W3 = None
        self.b1 = None
        self.b2 = None
        self.b3 = None
        self.A1 = None
        self.A2 = None  # sigmoid output of the second activation
        self.A3 = None
        self.learning_rate = learning_rate
        self.n_iterate = n_iterate
#         self.dic = dic
        self.dic = dic
    
    def initialize_parameters(self):
        self.W1 = np.random.randn(self.n_h, self.n_x) * 0.01
        self.b1 = np.zeros((self.n_h, 1))
        
        self.W2 = np.random.randn(self.n_h2, self.n_h) * 0.01
        self.b2 = np.zeros((self.n_h2, 1))
        
        self.W3 = np.random.randn(self.n_y, self.n_h2) * 0.01
        self.b3 = np.zeros((self.n_y, 1))
    
    
    def MSE(self,Y):
        cost = 0
        for i in range(len(Y)):
            for j in range(len(Y[0])):
                cost+=(Y[i][j]-self.A2[i][j])**2
        cost = cost/len(Y)
        return cost
        
    def relu(self, z):
        return z * (z > 0)
    
#     def softmax(self,z):
#         exps = np.exp(z-np.max(z,axis= 1,keepdims = True))
#         return exps/np.sum(exps,axis = 1, keepdims = True)
    
    

#     def softmax(self,A):  
#         expA = np.exp(A)
#         return expA / expA.sum(axis=1, keepdims=True)
    
    def sigmoid(self, z):
        return 1 / (1 + np.exp(-z))
    
    def sigmoid_derv(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))

    
    
    def cro_entro(self,Y):
        cost = - (Y * np.log(self.A2) + (1-Y) * np.log(1-self.A2)).mean()
        return cost
    
    def compute_cost(self, Y):# cross entrophy
        #cost = np.linalg.norm(self.A2 - Y)
        cost = - (Y * np.log(self.A3) + (1-Y) * np.log(1-self.A3)).mean()
#         print('shape cost is: ' ,cost.shape)
        return np.squeeze(cost) 

    def forward_propagation(self, X):
        self.A1 = self.relu(self.W1 @ X + self.b1)
        #dropout function for output in hidden layer
        u1 = np.random.binomial(1,0.3,size = self.A1.shape)
        self.A1 *= u1

        self.A2 = self.relu(self.W2 @ self.A1 + self.b2)
        #dropout function for hidden layer two
        u2 = np.random.binomial(1,0.3,size = self.A2.shape)
        self.A2 *= u2
        
        self.A3 = self.sigmoid(self.W3 @self.A2 + self.b3)
        

        
#         u2 = np.random.binomial(1,0.5,size = self.A2.shape)
#         self.A2 *= u2
#         self.A2 = self.soft_max_For_multi_attribute(self.A2,self.dic)

        
    
    def backward_propagation(self, X, Y):
        m = X.shape[1]  #num of input candidate

#         dZ2 = self.A2 - Y
# #         print('cur shape of dz2 :',dZ2.shape)
# #         print('cur cost of cros entro', c)
#         dW2 = dZ2 @ self.A1.T / m
#         db2 = np.sum(dZ2, axis=1, keepdims=True) / m
#         dZ1 = self.W2.T @ dZ2 * (self.A1 > 0)
#         dW1 = dZ1 @ X.T / m
#         db1 = np.sum(dZ1, axis=1, keepdims=True) / m

#         self.W1 -= self.learning_rate * dW1
#         self.b1 -= self.learning_rate * db1
#         self.W2 -= self.learning_rate * dW2
#         self.b2 -= self.learning_rate * db2

# <------------------------------------------------------>
#           used for two sigmoid hidden layer
#         dZ3 = (self.A3 - Y)/m
#         dW3 = np.dot(dZ3 , self.A2.T) 
#         db3 = np.sum(dZ3,axis = 1, keepdims = True)
#         dZ2 = np.dot(self.W3.T,  dZ3) * self.sigmoid_derv(self.A2)
#         dW2 = dZ2 @self.A1.T
#         db2 = np.sum(dZ2,axis = 1, keepdims = True)
#         dZ1 = np.dot(self.W2.T , dZ2) * self.sigmoid_derv(self.A1)
#         dW1 = dZ1@ X.T
#         db1 = np.sum(dZ1,axis = 1, keepdims = True)
# <------------------------------------------------------->


        dZ3 = (self.A3 - Y)/m
        dW3 = np.dot(dZ3 , self.A2.T) 
        db3 = np.sum(dZ3,axis = 1, keepdims = True)
        dZ2 = np.dot(self.W3.T,  dZ3) * (self.A2 > 0)
        dW2 = dZ2 @self.A1.T
        db2 = np.sum(dZ2,axis = 1, keepdims = True)
        dZ1 = np.dot(self.W2.T , dZ2) * (self.A1 > 0)
        dW1 = dZ1@ X.T
        db1 = np.sum(dZ1,axis = 1, keepdims = True)
        
        self.W1 -= self.learning_rate * dW1
        self.b1 -= self.learning_rate * db1
        self.W2 -= self.learning_rate * dW2
        self.b2 -= self.learning_rate * db2
        self.W3 -= self.learning_rate * dW3
        self.b3 -= self.learning_rate * db3
    
    def fit(self, X, Y):
        X, Y = X.T, Y.T
    
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
        self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                  self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))

    def fit_continue(self,X,Y):
        X, Y = X.T, Y.T
        self.n_x = X.shape[0]
        self.n_y = Y.shape[0]
#         if self.W1 is None:
#             self.initialize_parameters()

        # gradient descent
        for i in range(0, self.n_iterate):
            self.forward_propagation(X)
            self.backward_propagation(X, Y)
            if i % 10 == 0:
                cost = self.compute_cost(Y)
#                 self.learning_rate = 5 * cost
                print("Cost after iteration %i: %f" % (i, cost))
                
    def predict(self, X):
        X = X.T
        A1 = self.sigmoid(self.W1 @ X + self.b1)
        A2 = self.sigmoid(self.W2 @ A1 + self.b2)
        A3 = self.sigmoid(self.W3 @ A2 + self.b3)
        
        return A3.T
    
    def predict_one_layer(self,X):
        X = X.T
        A1 = self.relu(self.W1 @ X + self.b1)
        return A1.T
    
    def soft_max_For_multi_attribute(self,arr,dic):
        #create array with all dict keys based on insertion order
        tmp_arr = np.copy(arr)
        dicarr = list(dic.keys())
        cur_loc = 0
        cur_des = 0
        for j in range(len(dicarr)):
            cur_dict_key = dicarr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length    
            #find the targeted array need to perform softmax
            #create a dic to contain the location of those onehotencoding attribute
            loc_dic = []
            for i in range(cur_dict_length-1):
                loc_dic.append(cur_loc+i)
            #find targed array
            target = tmp_arr[:,loc_dic]
            #get softmaxed arr
            arr_soft_max = self.api_softmax(target)
            #reassig value back to orignal arr
            tmp_arr[:,loc_dic] = arr_soft_max
            #update cur_loc
            cur_loc +=cur_dict_length
            
        return tmp_arr

    def api_softmax(self,z):
        exps = np.exp(z)
        return exps/np.sum(exps,axis = 1, keepdims = True)

In [252]:
#create testing attribut arra
arr = []
og_arr = []
og_train = {} #init global dictionary
og_pred = {}
arr.append(['age',1])
arr.append(['mood_01',2]) #Today I generally feel
arr.append(['mood_02',2])
arr.append(['major',0])
arr.append(['big5_01',2]) #I see myself as: Extraverted, enthusiastic.
arr.append(['big5_02',2]) #I see myself as: Critical, quarrelsome.
arr.append(['big5_03',2]) #I see myself as: Dependable, self-disciplined.
arr.append(['big5_04',2]) #I see myself as: Anxious, easily upset.
arr.append(['big5_05',2]) #I see myself as: Open to new experiences, complex.
arr.append(['big5_06',2]) #I see myself as: Reserved, quiet.
arr.append(['big5_07',2]) #I see myself as: Sympathetic, warm.
arr.append(['big5_08',2]) #I see myself as: Disorganized, careless.
arr.append(['big5_09',2]) #I see myself as: Calm, emotionally stable.
arr.append(['big5_10',2]) #I see myself as: Conventional, uncreative.
arr.append(['highpower',0])
arr.append(['intrinsic_01',2])
arr.append(['intrinsic_02',2])
arr.append(['intrinsic_03',2])
arr.append(['intrinsic_04',2])
arr.append(['intrinsic_05',2])
arr.append(['intrinsic_06',2])
arr.append(['intrinsic_07',2])
arr.append(['intrinsic_08',2])
arr.append(['intrinsic_09',2])
arr.append(['intrinsic_10',2])
arr.append(['intrinsic_11',2])
arr.append(['intrinsic_12',2])
arr.append(['intrinsic_13',2])
arr.append(['intrinsic_14',2])
arr.append(['intrinsic_15',2])
arr.append(['lowpower',0])
arr.append(['mcfiller1',2])
arr.append(['mcfiller2',2])
arr.append(['mcfiller3',2])
arr.append(['mcmost1',2])
arr.append(['mcmost2',2])
arr.append(['mcmost3',2])
arr.append(['mcmost4',2])
arr.append(['mcmost5',2])
arr.append(['mcsome1',2])
arr.append(['mcsome2',2])
arr.append(['mcsome3',2])
arr.append(['mcsome4',2])
arr.append(['mcsome5',2])
arr.append(['mcdv1',2])
arr.append(['mcdv2',2])
arr.append(['pate_01',2])
arr.append(['pate_02',2])
arr.append(['pate_03',2])
arr.append(['pate_04',2])
arr.append(['pate_05',2])
arr.append(['stress_01',2])
arr.append(['stress_02',2])
arr.append(['stress_03',2])
arr.append(['stress_04',2])
arr.append(['nfc_01',2])
arr.append(['nfc_02',2])
arr.append(['nfc_03',2])
arr.append(['nfc_04',2])
arr.append(['nfc_05',2])
arr.append(['nfc_06',2])

In [253]:
# predict_df = df3.iloc[:100]
trainning_df = df3
# trainning_df = trainning_df.reset_index(drop = True)

# X_train = semi_auto_append_attri(arr,predict_df,og)
X_train = new_semi_auto_append_attri(arr,trainning_df,og_pred)
# print('start training dataset')
# X_tra = semi_auto_append_attri(arr,trainning_df,og_train)

cur key is age
cur key is mood_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 12)
cur key is mood_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 20)
cur key is major


/usr/lib/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:79: RuntimeWarning: Mean of empty slice.
/usr/lib/anaconda3/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


nan
class_arr of nlp [0, 2, 4, 6, 7, 9, 12, 13, 17, 20, 23, 27, 29]
(2998, 34)
cur key is big5_01
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 42)
cur key is big5_02
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 50)
cur key is big5_03
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 58)
cur key is big5_04
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 66)
cur key is big5_05
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 74)
cur key is big5_06
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 82)
cur key is big5_07
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 90)
cur key is big5_08
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 98)
cur key is big5_09
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 106)
cur key is big5_10
class_arr [1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0]
(2998, 114)
cur key is highpower
nan
class_arr of nlp [0, 2, 3, 4, 5, 7, 10, 13, 14, 17, 18, 20, 23, 25, 26, 29]
(2998, 131)
cur key is intrinsic_01
class_arr [1.0, 2.0, 3

In [254]:
X_train_new = X_train[:2900]
#slice nparr
x_real_train = X_train[:2900]
x_pred = X_train[2900:]

In [255]:
test = createlist_list_string(og_pred)
x,y = enlarge_dataset(X_train_new,X_train_new,og_pred,test)
dic_look = create_dictionary_for_every_location_of_feature(og_pred)

In [259]:
#write function to complete quesiton: Analyze the Data
#step1 : use the answer from evalue function - > those feature that has high dependencies
#first find those masked vector based on the name of features with high depen
#step2 : try also mask every other feature to which additional maskede feature lead to smallest accurayc
#of interpolating
def Analyzethedata(x,y,dic,arr_of_feature_high_mutual_dependencies,masked_arr,dic_location,hidden_neuron_num,iter_num):
    finalans = {}
    for i in range(len(arr_of_feature_high_mutual_dependencies)):
        for j in range(len(masked_arr)):
            if arr_of_feature_high_mutual_dependencies[i] == masked_arr[j][0]:
                cur_key_ans_arr = []
                #now current index j could be used to find the masked array
                x_feature_masked_high_muldependencies = x[j]
                y_feature_masked_high_muldependencies = y[j]
                cur_masked_key = arr_of_feature_high_mutual_dependencies[i]
                #step 2 loop throught every other key
                #first aplit dataset into training and preding
                for k in range(len(masked_arr)):
                    if masked_arr[k][0]!= cur_masked_key:
                        print('current masked key is', cur_masked_key, ' and other key is', masked_arr[k][0])
                        #masked other feature  -> first find the location of other feaure in one hot encoded array
                        x_cur_tmp_feature_masked_high_muldenpendencies = np.copy(x_feature_masked_high_muldependencies)
                        y_cur_tmp_feature_masked_high_muldenpendencies = np.copy(y_feature_masked_high_muldependencies)
                        cur_key = masked_arr[k][0]
                        #find location
                        cur_loc = dic_location.get(cur_key)[0]
                        cur_des = dic_location.get(cur_key)[1]
                        print('cur masked loc', cur_loc, 'cur des ', cur_des)
                        #mask
                        for pointer in range(len(x_cur_tmp_feature_masked_high_muldenpendencies)):
                            for pointer2 in range(cur_loc,cur_des):
                                x_cur_tmp_feature_masked_high_muldenpendencies[pointer][pointer2] = 0
                        #mask process complete
                        
                        #split dataset into training and predicting
                        x_cur_train = x_cur_tmp_feature_masked_high_muldenpendencies[:2000]
                        y_cur_train = y_cur_tmp_feature_masked_high_muldenpendencies[:2000]
                        x_cur_pred = x_cur_tmp_feature_masked_high_muldenpendencies[2000:]
                        y_cur_pred = y_cur_tmp_feature_masked_high_muldenpendencies[2000:]
                        nn = NeuralNetwork(hidden_neuron_num,iter_num,0.1,dic)
                        nn.fit(x_cur_train,y_cur_train)
                        #predict the asnwer
                        predict_ans = nn.predict(x_cur_pred)
                        totallen = len(predict_ans)
                        tmp_eval_ans = count_right_predicted_num(predict_ans,y_cur_pred,dic,cur_masked_key)
                        print('curmasked key is', cur_masked_key,'cur_iter_key is', cur_key,'percent is', tmp_eval_ans)
                        cur_dic = {}
                        cur_dic[cur_key] = tmp_eval_ans
                        cur_key_ans_arr.append(cur_dic)
                finalans[cur_masked_key] = cur_key_ans_arr = []
    return finalans

In [40]:
di = evaluate(x,y,og_pred,test,2000,200)

n_x is 334 n_y is  334
Cost after iteration 0: 0.693137
Cost after iteration 10: 0.608048
Cost after iteration 20: 0.366226
Cost after iteration 30: 0.321158
Cost after iteration 40: 0.288153
Cost after iteration 50: 0.275451
Cost after iteration 60: 0.262345
Cost after iteration 70: 0.256449
Cost after iteration 80: 0.247787
Cost after iteration 90: 0.243578
Cost after iteration 100: 0.237524
Cost after iteration 110: 0.232126
Cost after iteration 120: 0.227163
Cost after iteration 130: 0.223249
Cost after iteration 140: 0.218627
Cost after iteration 150: 0.212329
Cost after iteration 160: 0.210667
Cost after iteration 170: 0.204694
Cost after iteration 180: 0.199388
Cost after iteration 190: 0.193867
current loc and des is 0 and  4  total cound is 185 percent is 0.37
cur masked name is  age  eval value is:  0.37
n_x is 334 n_y is  334
Cost after iteration 0: 0.693338
Cost after iteration 10: 0.608776
Cost after iteration 20: 0.366608
Cost after iteration 30: 0.321556
Cost after itera

current loc and des is 68 and  76  total cound is 167 percent is 0.334
cur masked name is  big5_07  eval value is:  0.334
n_x is 334 n_y is  334
Cost after iteration 0: 0.693349
Cost after iteration 10: 0.602155
Cost after iteration 20: 0.368933
Cost after iteration 30: 0.321973
Cost after iteration 40: 0.288587
Cost after iteration 50: 0.277427
Cost after iteration 60: 0.260672
Cost after iteration 70: 0.254386
Cost after iteration 80: 0.246495
Cost after iteration 90: 0.242232
Cost after iteration 100: 0.237617
Cost after iteration 110: 0.233832
Cost after iteration 120: 0.227371
Cost after iteration 130: 0.220197
Cost after iteration 140: 0.213409
Cost after iteration 150: 0.215503
Cost after iteration 160: 0.205877
Cost after iteration 170: 0.205322
Cost after iteration 180: 0.200915
Cost after iteration 190: 0.189226
current loc and des is 76 and  84  total cound is 130 percent is 0.26
cur masked name is  big5_08  eval value is:  0.26
n_x is 334 n_y is  334
Cost after iteration 0:

Cost after iteration 150: 0.211830
Cost after iteration 160: 0.207758
Cost after iteration 170: 0.206549
Cost after iteration 180: 0.199966
Cost after iteration 190: 0.201031
current loc and des is 133 and  139  total cound is 222 percent is 0.444
cur masked name is  intrinsic_07  eval value is:  0.444
n_x is 334 n_y is  334
Cost after iteration 0: 0.692844
Cost after iteration 10: 0.608390
Cost after iteration 20: 0.365803
Cost after iteration 30: 0.323466
Cost after iteration 40: 0.290505
Cost after iteration 50: 0.283459
Cost after iteration 60: 0.264228
Cost after iteration 70: 0.257194
Cost after iteration 80: 0.246894
Cost after iteration 90: 0.240349
Cost after iteration 100: 0.236990
Cost after iteration 110: 0.230340
Cost after iteration 120: 0.228695
Cost after iteration 130: 0.224564
Cost after iteration 140: 0.218090
Cost after iteration 150: 0.214751
Cost after iteration 160: 0.210779
Cost after iteration 170: 0.202022
Cost after iteration 180: 0.199342
Cost after iteratio

Cost after iteration 90: 0.253943
Cost after iteration 100: 0.235575
Cost after iteration 110: 0.232020
Cost after iteration 120: 0.227651
Cost after iteration 130: 0.222736
Cost after iteration 140: 0.216552
Cost after iteration 150: 0.211640
Cost after iteration 160: 0.204418
Cost after iteration 170: 0.203054
Cost after iteration 180: 0.197679
Cost after iteration 190: 0.191775
current loc and des is 187 and  193  total cound is 265 percent is 0.53
cur masked name is  mcfiller2  eval value is:  0.53
n_x is 334 n_y is  334
Cost after iteration 0: 0.693167
Cost after iteration 10: 0.604700
Cost after iteration 20: 0.363842
Cost after iteration 30: 0.320261
Cost after iteration 40: 0.287123
Cost after iteration 50: 0.280987
Cost after iteration 60: 0.261228
Cost after iteration 70: 0.251362
Cost after iteration 80: 0.244916
Cost after iteration 90: 0.242042
Cost after iteration 100: 0.234255
Cost after iteration 110: 0.232791
Cost after iteration 120: 0.225783
Cost after iteration 130:

Cost after iteration 50: 0.279033
Cost after iteration 60: 0.261217
Cost after iteration 70: 0.253278
Cost after iteration 80: 0.247042
Cost after iteration 90: 0.243837
Cost after iteration 100: 0.236794
Cost after iteration 110: 0.237149
Cost after iteration 120: 0.227932
Cost after iteration 130: 0.222578
Cost after iteration 140: 0.217393
Cost after iteration 150: 0.218502
Cost after iteration 160: 0.209406
Cost after iteration 170: 0.207284
Cost after iteration 180: 0.203524
Cost after iteration 190: 0.196796
current loc and des is 222 and  225  total cound is 140 percent is 0.28
cur masked name is  mcsome4  eval value is:  0.28
n_x is 334 n_y is  334
Cost after iteration 0: 0.692947
Cost after iteration 10: 0.613454
Cost after iteration 20: 0.370950
Cost after iteration 30: 0.330205
Cost after iteration 40: 0.289781
Cost after iteration 50: 0.268393
Cost after iteration 60: 0.259171
Cost after iteration 70: 0.253818
Cost after iteration 80: 0.247480
Cost after iteration 90: 0.246

Cost after iteration 10: 0.602379
Cost after iteration 20: 0.366447
Cost after iteration 30: 0.318851
Cost after iteration 40: 0.285729
Cost after iteration 50: 0.281740
Cost after iteration 60: 0.260108
Cost after iteration 70: 0.252315
Cost after iteration 80: 0.244432
Cost after iteration 90: 0.243589
Cost after iteration 100: 0.235165
Cost after iteration 110: 0.229579
Cost after iteration 120: 0.224059
Cost after iteration 130: 0.225406
Cost after iteration 140: 0.220312
Cost after iteration 150: 0.217616
Cost after iteration 160: 0.204355
Cost after iteration 170: 0.208463
Cost after iteration 180: 0.197274
Cost after iteration 190: 0.192505
current loc and des is 280 and  286  total cound is 205 percent is 0.41
cur masked name is  stress_02  eval value is:  0.41
n_x is 334 n_y is  334
Cost after iteration 0: 0.693432
Cost after iteration 10: 0.604937
Cost after iteration 20: 0.367456
Cost after iteration 30: 0.321406
Cost after iteration 40: 0.287767
Cost after iteration 50: 0.2

# Answer Question in Analyze the Data

In [260]:
# a = ['mood_02,','intrinsic_01','intrinsic_02','intrinsic_03','intrinsic_04'
#     ,'intrinsic_08','intrinsic_10','intrinsic_13','intrinsic_15','mcfiller1'
#      ,'mcfiller2','mcfiller3','mcfiller1','mcfiller1','mcfiller1','mcfiller2'
#      ,'mcfiller3','mcdv1','pate_03','pate_04','stress_02','stress_02','stress_03'
#      ,'nfc_02','nfc_04','nfc_05'
#     ]
a = ['mood_02']

In [ ]:
final = Analyzethedata(x,y,og_pred,a,test,dic_look,2000,70)

current masked key is mood_02  and other key is age
cur masked loc 0 cur des  4
n_x is 374 n_y is  374
Cost after iteration 0: 0.693200
Cost after iteration 10: 0.530341
Cost after iteration 20: 0.334486
Cost after iteration 30: 0.291431
Cost after iteration 40: 0.261025
Cost after iteration 50: 0.247439
Cost after iteration 60: 0.238016
current loc and des is 12 and  20  total cound is 311 percent is 0.34555555555555556
curmasked key is mood_02 cur_iter_key is age percent is 0.34555555555555556
current masked key is mood_02  and other key is mood_01
cur masked loc 4 cur des  12
n_x is 374 n_y is  374
Cost after iteration 0: 0.693074
Cost after iteration 10: 0.498517
Cost after iteration 20: 0.404013
Cost after iteration 30: 0.292024
Cost after iteration 40: 0.259768
Cost after iteration 50: 0.248208
Cost after iteration 60: 0.239170
current loc and des is 12 and  20  total cound is 302 percent is 0.33555555555555555
curmasked key is mood_02 cur_iter_key is mood_01 percent is 0.3355555

Cost after iteration 0: 0.693037
Cost after iteration 10: 0.412716
Cost after iteration 20: 0.360389
Cost after iteration 30: 0.305250
Cost after iteration 40: 0.262731
Cost after iteration 50: 0.249000
Cost after iteration 60: 0.243362
current loc and des is 12 and  20  total cound is 302 percent is 0.33555555555555555
curmasked key is mood_02 cur_iter_key is intrinsic_03 percent is 0.33555555555555555
current masked key is mood_02  and other key is intrinsic_04
cur masked loc 147 cur des  153
n_x is 374 n_y is  374
Cost after iteration 0: 0.693265
Cost after iteration 10: 0.432638
Cost after iteration 20: 0.449749
Cost after iteration 30: 0.287413
Cost after iteration 40: 0.259366
Cost after iteration 50: 0.249190
Cost after iteration 60: 0.241948
current loc and des is 12 and  20  total cound is 302 percent is 0.33555555555555555
curmasked key is mood_02 cur_iter_key is intrinsic_04 percent is 0.33555555555555555
current masked key is mood_02  and other key is intrinsic_05
cur maske

# TODO : CREATE FUNCTION TO RECOVER CLASSIFICATION DATASET INTO HUMAN-Fridendly DATASET

In [ ]:
def recover_answer(predict_arr,dic):
    reverse_dic = create_reverse_dictionary(dic)
    #init an dic as final anser array
    ans_arr = []
    #create array with all dict keys based on insertion order
    dic_arr = list(dic.keys())
    #recover predicted candidate sequentilaay
    for i in range(len(predict_arr)):
        tmp_ans_array = {} # cur dict to store anser for this candidate
        cur_loc = 0
        cur_des = 0
        #loop through dictionary array to recover each attribute column
        for j in range(len(dic_arr)):
            cur_dict_key = dic_arr[j]
            cur_dict_arr = dic.get(cur_dict_key)
            cur_reverse_dic_arr = reverse_dic.get(cur_dict_key)
            cur_dict_length = len(cur_dict_arr)+1
            cur_des += cur_dict_length #update current cutting position
            #restore
            tmp_cutting_arr = predict_arr[i][cur_loc:cur_des]
            tag = tmp_cutting_arr[-1]
            if tag == 0:
                tmp_ans_array[cur_dict_key] = 'NA'
            else:
                #find the position where tag is 1
                loc = 0
                for pos in range(len(tmp_cutting_arr)):
                    if tmp_cutting_arr[pos] == 1:
                        loc = pos
                        break
                #now based on location, create cover anser
                #case one loc is 0
                if loc == 0:
                    ans ="<" + str(cur_reverse_dic_arr.get(loc))
                    tmp_ans_array[cur_dict_key] = ans
                else:
                    last_loc = loc-1
                    ans=""
                    ans+=str(cur_reverse_dic_arr.get(last_loc))+" < "
                    ans+=str(cur_reverse_dic_arr.get(loc))
                    tmp_ans_array[cur_dict_key] = ans
#                 ans =cur_reverse_dic_arr.get(loc)
#                 tmp_ans_array[cur_dict_key] = ans
        
            cur_loc+=cur_dict_length
        #append current tmp candidate array to finaly answer
        ans_arr.append(tmp_ans_array)
    
    return ans_arr
                    

# CREATE FUNCTION FOR CROSS VALIDATION
Assess the accuracy by count how many

In [ ]:
def cross_validation(arr_all_candidate,num_fold,og_pred,size_hidden_layer,iter_num,learning_rate):
    cost = 0
    #create num_fold equal size fold from input nparray
    list_of_fold = np.array_split(arr_all_candidate,num_fold)
    
    #need to append training dataset together, while leave testing set alone
    for i in range(num_fold):
        x_pred = list_of_fold[i]
        
        tmp = 0
        for j in range(0,num_fold):
            if j == i:
                continue
            else:
                
                if tmp == 0:
                    ans = np.asarray(list_of_fold[j])
                    tmp+=1
                
                elif tmp > 0:
                    
                    ans = cro_new_append(ans,list_of_fold[j])

            
        #finished rebuilding training dataset
        #start training neural network
        nn = NeuralNetwork(size_hidden_layer,iter_num,learning_rate) #size of input layer
        
        nn.fit(ans,ans)
        predict_ans = nn.predict(x_pred)
        find_classification_based_on_distance(predict_ans,og_pred)
        #coun
        tmpcost = cost_function(predict_ans,x_pred)
        cost+=tmpcost
        print('#### current cutting position is : ', i, 'current cost is: ',tmpcost)
    print('final avg cost is : ', cost/num_fold)
    return cost/num_fold



def cost_function(x,y):
    #compute the average num of wrongly predicted column as cost function
    total_num = len(x)
    count = 0
    
    for i in range(len(x)):
        tmpcount = 0
        for j in range(len(x[i])):
            if x[i][j]!=y[i][j]:
                tmpcount+=1
        count+=tmpcount
    avg_count = count/total_num
    return avg_count
def cross_valid_append_arr(old,new):
    ans = np.append([old],[new],axis = 0)
    return ans

def cro_new_append(old,new):
    #num_col
    num_col = len(old[0])
    new_ans = []
    for i in range(len(old)):
        new_ans.append(old[i])
    for i in range(len(new)):
        new_ans.append(new[i])
    return np.asarray(new_ans)

# TEST CROSS VALIDATION